In [137]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import random

In [138]:
x = torch.rand([1,2,2])
print(x)

tensor([[[0.2381, 0.2836],
         [0.7001, 0.2369]]])


In [161]:
class Module_conv_bn(nn.Module):
    
    def __init__(self) -> None:
        super(Module_conv_bn, self).__init__()
        self.conv = nn.Conv2d(1, 3, kernel_size=2, padding=1, stride=1, bias=True)
        self.batch = nn.BatchNorm1d(3)
    
    def forward(self, x):
        x = self.conv(x)
        #
        mean_x = x.sum()/sum(x.size())
        std = torch.sqrt(x.var())
        print(mean_x, std)
        #
        x = self.batch(x)
        return x, self.conv.weight.data, self.conv.bias.data, self.batch.weight.data, mean_x, std

In [214]:
mod = Module_conv_bn()
res, wt_conv, bs_conv, wt_batch, mean_x, std = mod(x)

print(res)

tensor(0.3928, grad_fn=<DivBackward0>) tensor(0.2971, grad_fn=<SqrtBackward0>)
tensor([[[ 1.0097,  1.4177,  0.9557],
         [ 1.2351,  1.3143,  0.6161],
         [-0.2249,  0.2097,  0.3279]],

        [[ 0.3148, -0.0903,  0.1503],
         [-0.1550, -0.1916,  0.4695],
         [ 0.8642,  1.6385,  0.6893]],

        [[-0.9880, -1.1471, -1.6230],
         [-0.1209, -1.4190, -1.7485],
         [-0.4399, -1.5480, -1.5167]]], grad_fn=<NativeBatchNormBackward0>)


In [237]:
class Only_Conv(nn.Module):
    def __init__(self) -> None:
        super(Only_Conv, self).__init__()
        self.new_conv = nn.Conv2d(1, 3, kernel_size=2, padding=1, stride=1, bias=True)
        # I need to update date for that conv
        # w=
        self.new_conv.weight.data = (wt_conv * wt_batch[0])/ std 
        # bias = 
        self.new_conv.bias.data = ((bs_conv * wt_batch[0])/ std) + mean_x

    def forward(self, x):
        x = self.new_conv(x)
        return x

In [238]:
new_mod = Only_Conv()
res = new_mod(x)
print(res.tolist())

[[[1.0097885131835938, 1.4178141355514526, 0.9558128118515015], [1.2351500988006592, 1.3143903017044067, 0.6160908937454224], [-0.22495843470096588, 0.20967090129852295, 0.3279365599155426]], [[0.3148476481437683, -0.09028516709804535, 0.15029263496398926], [-0.15498578548431396, -0.1915912628173828, 0.4695455729961395], [0.864259660243988, 1.638581395149231, 0.6893236637115479]], [[-0.988045871257782, -1.1471439599990845, -1.623081088066101], [-0.12095446139574051, -1.4190393686294556, -1.7486064434051514], [-0.4399503171443939, -1.5481081008911133, -1.51675546169281]]]
